# Extract data from database
I only extract relations whose name starts with course or users according to Miguel's suggestion.

In [1]:
import psycopg2
import psycopg2.extras
import pandas as pd

### Database configuration

In [2]:
dbname = "pps_prd"
user = "ppsuser"
password = "jF2RwCuTgbXrKMb]$fxUVCpGD9Aw"
host = "machine-learning.cq0an2uihaqt.us-east-1.rds.amazonaws.com"

### Define a function for copying data from database to local csv files.

In [25]:
def copyToFile():
    tableList = []
    count = 0
    conn = psycopg2.connect(dbname = dbname,user = user, password = password, host = host)
    curSelectTable = conn.cursor(cursor_factory=psycopg2.extras.DictCursor)
    curSelectTable.execute("select relname,reltuples from pg_class where relkind='r' and relname !~ '^(pg_|sql_)';")
    for table in curSelectTable.fetchall():
        if(table[1]>0):
            tName = str(table[0])
            #curCheckNull = conn.cursor()
            #queryCheckNull = "SELECT EXISTS(SELECT 1 FROM pg_class WHERE relname=\'"+table+"\');"
            #curCheckNull.execute(queryCheckNull)
            #if curCheckNull.fetchone()[0] == True:
            if("course" in tName or "user" in tName):
                count += 1
                tableList.append(tName)
                curCopyTo = conn.cursor()
                filePath = "data/"+tName+".csv" #need change the path
                queryCopy = "COPY %s TO STDOUT WITH CSV HEADER DELIMITER AS ',';"
                with open(filePath, 'w') as f:
                    #curCopyTo.copy_expert(sql=queryCopy % tName,file=f)
                    print (tName +" is done.")
    print (count, " files are copied.")
    return tableList
    conn.close()

### Define a function to explore table relationships by showing foreign keys
Output a list.
<h5>SQL Statement</h5>
<p>SELECT</p>
   tc.table_name, tc.constraint_name, kcu.column_name, <br></br>
    ccu.table_name AS foreign_table_name,<br></br>
    ccu.column_name AS foreign_column_name <br></br>
<p>FROM</p>
    information_schema.table_constraints AS tc <br></br>
    JOIN information_schemaa.key_column_usage AS kcu<br></br>
      ON tc.constraint_name = kcu.constraint_name<br></br>
    JOIN information_schema.constraint_column_usage AS ccu //the columns that the foreign key references. <br></br>
      ON ccu.constraint_name = tc.constraint_name<br></br>
<p>WHERE</p>
    constraint_type = 'FOREIGN KEY' AND tc.table_name ~ '^(course|user)';



In [55]:
def showFK(tableList=[]):
    conn = psycopg2.connect(dbname = dbname,user = user, password = password, host = host)
    curFK = conn.cursor(cursor_factory=psycopg2.extras.DictCursor)
    # select foriegn key
    queryFK = "SELECT tc.table_name,tc.constraint_name,kcu.column_name, ccu.table_name AS foreign_table_name,ccu.column_name AS foreign_column_name FROM information_schema.table_constraints AS tc JOIN information_schema.key_column_usage AS kcu ON tc.constraint_name = kcu.constraint_name JOIN information_schema.constraint_column_usage AS ccu ON ccu.constraint_name = tc.constraint_name WHERE constraint_type = 'FOREIGN KEY'"
    """if len(tableList)>0:
        for t in tableList:
            queryFK += (" AND (tc.table_name = '%s' OR ccu.table_name = '%s');" %(t,t))
            curFK.execute(queryFK)
    else:"""
    queryFK += " AND tc.table_name ~ '^(course|user)' ;"
    curFK.execute(queryFK)
    listFK = []
    count = 0
    for i in curFK:
        listFK.append(i[:])
        #print(i)
        count+=1
    print(count,"relationships found.")
    return listFK
    conn.close()


In [26]:
tableList = copyToFile()

course_unit_articles is done.
course_resource is done.
course_unit_resources is done.
course_classroom is done.
course_article_questions is done.
course_course is done.
user_app_address is done.
user_app_customuser_user_permissions is done.
user_app_customuser is done.
user_messages_room_subscribers is done.
course_video is done.
course_article is done.
user_messages_message is done.
course_course_units is done.
user_messages_room_new_message is done.
user_messages_room is done.
course_progress is done.
course_unit is done.
course_match is done.
course_subquestionresponse is done.
course_match_student_match is done.
course_questionresponse_new_subresponse is done.
course_question is done.
course_questionresponse_new_response is done.
course_questionresponse is done.
25  files are copied.


Totally 25 files are copied to local.

In [56]:
listFK=showFK()

40 relationships found.


In [12]:
df=pd.DataFrame(listFK,columns=["tableName","constraint","column","foreignTable","foreignColunm"])
df.drop('constraint', axis=1, inplace=True)
df.head()

,tableName,column,foreignTable,foreignColunm
0,user_app_customuser_user_permissions,permission_id,auth_permission,id
1,user_app_customuser_groups,group_id,auth_group,id
2,user_app_customuser_groups,customuser_id,user_app_customuser,id
3,user_app_customuser_user_permissions,customuser_id,user_app_customuser,id
4,user_messages_room_subscribers,customuser_id,user_app_customuser,id


In [60]:
listCU = []
for i in listFK:
    if i[0] in tableList and i[3] in tableList:
        listCU.append(i)
print(len(listCU))
        

37


In [61]:
dfCU=pd.DataFrame(listCU,columns=["tableName","constraint","column","foreignTable","foreignColunm"])
dfCU.drop('constraint', axis=1, inplace=True)
dfCU.head(37)

,tableName,column,foreignTable,foreignColunm
0,user_app_customuser_user_permissions,customuser_id,user_app_customuser,id
1,user_messages_room_subscribers,customuser_id,user_app_customuser,id
2,user_messages_room_new_message,customuser_id,user_app_customuser,id
3,user_messages_message,sender_id,user_app_customuser,id
4,course_classroom,teacher_id,user_app_customuser,id
5,course_match,student_id,user_app_customuser,id
6,user_app_customuser,primary_address_id,user_app_address,id
7,course_article_questions,question_id,course_question,id
8,course_questionresponse,question_id,course_question,id
9,course_article,video_one_id,course_video,id


In [20]:
# exercise for copy expert
SQL_STATEMENT = "COPY course_question TO STDOUT WITH CSV HEADER DELIMITER AS ',';"
conn = psycopg2.connect(dbname = dbname,user = user, password = password, host = host)
cursor = conn.cursor()
tName='course_question'
with open("course_question.csv", 'w+') as f:
    cursor.copy_expert(sql=SQL_STATEMENT, file=f)
    #cursor.copy_to(f,tName)
cursor.close()

In [5]:
# exercise
conn = psycopg2.connect(dbname = dbname,user = user, password = password, host = host)
cursor = conn.cursor(cursor_factory=psycopg2.extras.DictCursor)

'''
# show table name of all the tables
cursor.execute("SELECT EXISTS(SELECT table_name FROM information_schema.tables WHERE table_type='BASE TABLE' AND table_name!~ '^(pg_|sql_)');")
#cursor.execute("select relname,reltuples from pg_class where relkind='r' and relname = 'auth_group_permissions';")

# show colunm names of al the tables
cursor.execute("select column_name from information_schema.columns where table_name = 'auth_group_permissions';")
column_names = [row[0] for row in cursor]
print column_names
'''


'\n# show table name of all the tables\ncursor.execute("SELECT EXISTS(SELECT table_name FROM information_schema.tables WHERE table_type=\'BASE TABLE\' AND table_name!~ \'^(pg_|sql_)\');")\n#cursor.execute("select relname,reltuples from pg_class where relkind=\'r\' and relname = \'auth_group_permissions\';")\n\n# show colunm names of al the tables\ncursor.execute("select column_name from information_schema.columns where table_name = \'auth_group_permissions\';")\ncolumn_names = [row[0] for row in cursor]\nprint column_names\n'